In [152]:
#r "nuget: Microsoft.ML.AutoML, 0.21.0-preview.23266.6"
#r "nuget: Microsoft.Data.Analysis, 0.23.0-preview.1.25127.4"
#r "nuget: Plotly.NET.Interactive, 5.0.0"

Installed Packages Microsoft.Data.Analysis, 0.23.0-preview.1.25127.4 Microsoft.ML.AutoML, 0.21.0-preview.23266.6 Plotly.NET.Interactive, 5.0.0

In [153]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;
using Plotly.NET;

using System.Threading;
using System.IO;
using System.Linq;
using Microsoft.Data.Analysis;
using System.Globalization;

In [154]:
public class SalesPrediction
{
    [ColumnName("Score")]
    public float PredictedSales { get; set; }
}
public class CarSalesData
{
    public float Year { get; set; }
    public float Month { get; set; }
    public float Sales { get; set; }
}

In [155]:
//Loads the model
var ctx = new MLContext();
var modelLoaded = ctx.Model.Load("./model.mlnet", out var modelSchema);

In [156]:
//Creates the prediction engine
var predEngine = ctx.Model.CreatePredictionEngine<CarSalesData, SalesPrediction>(modelLoaded);

In [157]:
//Use prediction engine to predict the sales for the next 12 months
var lastPrev = carSalesData.OrderBy(_ => _.Month).OrderBy(_ => _.Year).LastOrDefault();
var prevList = new List<CarSalesData>(){
    lastPrev
};

for (int i = 1; i < 12; i++)
{
    var month = lastPrev.Month + i;
    var year = lastPrev.Year;
    if(month > 12)
    {
        month = month - 12;
        year++;
    }

    var sample = new CarSalesData()
    {
        Year = year,
        Month = month,
    };
    var prediction = predEngine.Predict(sample);
    sample.Sales = prediction.PredictedSales;
    prevList.Add(sample);
}


// var sample = new CarSalesData()
// {
//     Year = 2027,
//     Month = 01,
// };
// var prediction = predEngine.Predict(sample);
// sample.Sales = prediction.PredictedSales;
// Console.WriteLine($"Previsão: {prediction.PredictedSales} unidades");

Error: (4,5): error CS1950: The best overloaded Add method 'List<CarSalesData>.Add(CarSalesData)' for the collection initializer has some invalid arguments
(4,5): error CS1503: Argument 1: cannot convert from 'CarSalesData [ℛ*8f7bc348-9065-40f2-a9c2-f5f94dc098b5#1-129, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null]' to 'CarSalesData [ℛ*8f7bc348-9065-40f2-a9c2-f5f94dc098b5#1-137, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null]'

In [158]:
// Define data path
var lines = File.ReadAllLines(@"./bcdata.sgs.1378.csv");
IEnumerable<CarSalesData> carSalesData = new List<CarSalesData>();
for (int i = 1; i < lines.Length; i++) // pula o cabeçalho
{
    var carSale = new CarSalesData();
    var parts = lines[i].Split(';');
    parts[0] = parts[0].Replace("\"",""); // remove aspas
    var date= DateTime.ParseExact(parts[0], "dd/MM/yyyy", CultureInfo.InvariantCulture);
    carSale.Year = date.Year;
    carSale.Month = date.Month;
    carSale.Sales = float.Parse(parts[1].Replace(",", ".").Replace("\"",""));
    carSalesData = carSalesData.Append(carSale);
}
Console.WriteLine($"Dados carregados: {carSalesData.Count()} registros.");

Dados carregados: 529 registros.


In [159]:
//Show and save data
var scatter = Chart2D.Chart.Scatter<DateTime, float, string>(
    x: carSalesData.Select(x => new DateTime((int)x.Year, (int)x.Month, 1)),
    y: carSalesData.Select(x => x.Sales),
    mode: StyleParam.Mode.Lines,
    Name: "Sales"
);


var scatterPrevision = Chart2D.Chart.Line<DateTime, float, string>(
    x: prevList.Select(x => new DateTime((int)x.Year, (int)x.Month, 1)),
    y: prevList.Select(x => x.Sales),
    LineColor: Color.fromString("red"),
    Name: "Prediction"
);
var combined = Chart.Combine(new []{scatter, scatterPrevision});

combined.Show();
combined.SaveHtml("graph.html");